In [1]:
import os,glob
import numpy as np
import SimpleITK as sitk
from math import fabs, copysign
from collections import OrderedDict
import shutil
from utils import *

In [2]:
data_file = 'F:/data/CellPress1338'

In [4]:
data_list = sorted(os.listdir(os.path.join(data_file,'cta_img')))
print (len(data_list))
print (data_list[0])

ts_data_list = []
for name in data_list:
    ts_data_list.append(name)
#print (ts_data_list)

1338
Tr0001.nii.gz


In [5]:
def generate_datainfo_csv(name,IA_voxels,slices):
 
    #name_attribute = ['id','slices','IA_voxels','IA_minz','IA_size']

    img = dict()
    img['id'] = name
    img['slices'] = slices
    img['IA_voxels'] = IA_voxels
    #img['IA_minz'] = IA_minz
    
    x = img['IA_voxels']
    y = copysign(fabs(x) ** (1 / 3), x)
    
    img['IA_size'] = int(y)
    
    return img

In [6]:
def generate_datainfo_pkl(name,cta,seg,data_file):

    property = OrderedDict()
    property['id'] = name
    property['ori_cta'] = os.path.join(data_file, 'cta_img', name + '.nii.gz')
    property['ori_seg'] = os.path.join(data_file, 'ane_seg', name + '.nii.gz')
    
    data_itk = cta
    property["original_size_of_raw_data"] = np.array(data_itk.GetSize())[[2, 1, 0]]
    property["original_spacing"] = np.array(data_itk.GetSpacing())[[2, 1, 0]]
    property["itk_origin"] = data_itk.GetOrigin()
    property["itk_spacing"] = data_itk.GetSpacing()
    property["itk_direction"] = data_itk.GetDirection()
    
    seg_np = sitk.GetArrayFromImage(seg).astype(np.int32)
    property['slices'] = seg_np.shape[0]
    property['IA_voxels'] = seg_np.sum()
    #IA_minz = np.min(np.where(seg_np==1)[0])
    #property['IA_minz'] = int(IA_minz)

    return property,property['IA_voxels'],property['slices']


In [7]:
csv_info_list = []
all_properties = OrderedDict()

for data in ts_data_list:
    name = data[:-7]
    print ('name:',name)
    cta_itk = sitk.ReadImage(os.path.join(data_file,'cta_img',data))
    seg_itk = sitk.ReadImage(os.path.join(data_file,'ane_seg',data))
    
    #pkl
    properti,IA_voxels,slices = generate_datainfo_pkl(name,cta_itk,seg_itk,data_file)
    #print ('IA_minz:',IA_minz)
    print ('slices:',slices)
    print ()
    all_properties[name] = properti
    
    
    #csv
    csv_info_list.append(generate_datainfo_csv(name,IA_voxels,slices))
    


name: Tr0001
slices: 392

name: Tr0002
slices: 817

name: Tr0003
slices: 446

name: Tr0004
slices: 413

name: Tr0005
slices: 405

name: Tr0006
slices: 412

name: Tr0007
slices: 921

name: Tr0008
slices: 791

name: Tr0009
slices: 419

name: Tr0010
slices: 843

name: Tr0011
slices: 412

name: Tr0012
slices: 908

name: Tr0013
slices: 406

name: Tr0014
slices: 805

name: Tr0015
slices: 900

name: Tr0016
slices: 422

name: Tr0017
slices: 434

name: Tr0018
slices: 828

name: Tr0019
slices: 908

name: Tr0020
slices: 876

name: Tr0021
slices: 856

name: Tr0022
slices: 866

name: Tr0023
slices: 833

name: Tr0024
slices: 856

name: Tr0025
slices: 856

name: Tr0026
slices: 768

name: Tr0027
slices: 841

name: Tr0028
slices: 497

name: Tr0029
slices: 418

name: Tr0030
slices: 828

name: Tr0031
slices: 449

name: Tr0032
slices: 495

name: Tr0033
slices: 908

name: Tr0034
slices: 843

name: Tr0035
slices: 495

name: Tr0036
slices: 942

name: Tr0037
slices: 796

name: Tr0038
slices: 803

name: Tr0039

In [8]:
# save csv
name_attribute = ['id','slices','IA_voxels','IA_size']
save_csv(csv_info_list,'All_CellPressAll_info.csv',name_attribute)


# save pkl
save_pickle(all_properties,'All_CellPressAll_info.pkl')